### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

### Preliminary Data 

In [5]:
df = pd.read_csv('./DATA/forestfinal.csv')

In [6]:
df.head()

,Unnamed: 0,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month,day
0,0,1.001657,0.569075,-0.801628,-1.322179,-1.837925,-0.858350,-1.835573,0.406971,1.498430,-0.073555,-0.201967,3,5
1,1,1.001657,-0.244116,-0.006761,-1.178717,0.484659,-0.508323,-0.152432,-0.694428,-1.733018,-0.073555,-0.201967,10,2
2,2,1.001657,-0.244116,-0.006761,-1.049290,0.556583,-0.508323,-0.736379,-0.694428,-1.510159,-0.073555,-0.201967,10,6
3,5,1.433101,1.382265,0.300346,-0.400591,-0.247109,1.241810,0.568915,-0.939183,0.774140,-0.073555,-0.201967,8,7
4,6,1.433101,1.382265,0.300346,-0.344454,-0.216399,-0.114543,0.895238,-1.061561,-0.507296,-0.073555,-0.201967,8,1


In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month,day
0,1.001657,0.569075,-0.801628,-1.322179,-1.837925,-0.858350,-1.835573,0.406971,1.498430,-0.073555,-0.201967,3,5
1,1.001657,-0.244116,-0.006761,-1.178717,0.484659,-0.508323,-0.152432,-0.694428,-1.733018,-0.073555,-0.201967,10,2
2,1.001657,-0.244116,-0.006761,-1.049290,0.556583,-0.508323,-0.736379,-0.694428,-1.510159,-0.073555,-0.201967,10,6
3,1.433101,1.382265,0.300346,-0.400591,-0.247109,1.241810,0.568915,-0.939183,0.774140,-0.073555,-0.201967,8,7
4,1.433101,1.382265,0.300346,-0.344454,-0.216399,-0.114543,0.895238,-1.061561,-0.507296,-0.073555,-0.201967,8,1


#### Split into Features and Target

In [9]:
X = df.drop('area', axis=1)

In [10]:
y = df['area']

In [11]:
X.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,month,day
0,1.001657,0.569075,-0.801628,-1.322179,-1.837925,-0.858350,-1.835573,0.406971,1.498430,-0.073555,3,5
1,1.001657,-0.244116,-0.006761,-1.178717,0.484659,-0.508323,-0.152432,-0.694428,-1.733018,-0.073555,10,2
2,1.001657,-0.244116,-0.006761,-1.049290,0.556583,-0.508323,-0.736379,-0.694428,-1.510159,-0.073555,10,6
3,1.433101,1.382265,0.300346,-0.400591,-0.247109,1.241810,0.568915,-0.939183,0.774140,-0.073555,8,7
4,1.433101,1.382265,0.300346,-0.344454,-0.216399,-0.114543,0.895238,-1.061561,-0.507296,-0.073555,8,1


### Train-Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Linear Regression

In [15]:
linear_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('linear', LinearRegression())
])

In [31]:
param_grid_lin = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'linear__fit_intercept': [True, False],
    'linear__positive': [True, False]
}

In [32]:
grid_search_lin = GridSearchCV(linear_pipeline, param_grid_lin, cv=5, scoring='r2')

In [33]:
grid_search_lin.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__fit_intercept': [True, False],
                         'linear__positive': [True, False],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [36]:
best_model_lin = grid_search_lin.best_estimator_

In [37]:
y_lin_preds = best_model_lin.predict(X_test)

### Polynomial Regression

In [40]:
poly_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('poly', PolynomialFeatures()), 
    ('linear', LinearRegression())
])

In [41]:
param_grid_poly = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()], 
    'poly__degree': [2,3,4], 
    'poly__interaction_only': [False, True],
    'linear__fit_intercept': [True, False], 
    'linear__positive': [True, False]
}

In [42]:
grid_poly = GridSearchCV(poly_pipeline, param_grid_poly, cv=5, scoring='r2')

In [43]:
grid_poly.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
11 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fi

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__fit_intercept': [True, False],
                         'linear__positive': [True, False],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()]},
             scoring='r2')

In [44]:
best_model_poly = grid_poly.best_estimator_

In [45]:
y_pred_poly = best_model_poly.predict(X_test)

### Ridge Regression

In [48]:
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

In [49]:
param_grid_ridge = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'poly__degree': [2,3,4],
    'poly__interaction_only': [False, True], 
    'ridge__alpha': [0.1, 1.0, 10, 100],
    'ridge__fit_intercept': [True, False]
}

In [50]:
ridge_grid = GridSearchCV(ridge_pipe, param_grid_ridge, cv=5, scoring='r2')

In [51]:
ridge_grid.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('ridge', Ridge())]),
             param_grid={'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'ridge__alpha': [0.1, 1.0, 10, 100],
                         'ridge__fit_intercept': [True, False],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [52]:
best_model_ridge = ridge_grid.best_estimator_

In [53]:
y_pred_ridge = best_model_ridge.predict(X_test)

### Lasso Regression

In [57]:
lasso_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('lasso', Lasso())
])

In [58]:
param_lasso = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'poly__degree': [2,3,4], 
    'poly__interaction_only': [False, True],
    'lasso__alpha': [0.001,0.01,0.1,1,10],
    'lasso__fit_intercept': [True,False],
}

In [59]:
grid_lasso = GridSearchCV(lasso_pipe, param_lasso, cv=5, scoring='r2')

In [60]:
grid_lasso.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.708e-02, tolerance: 4.639e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.570e-02, tolerance: 4.404e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.546e-02, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.318e+00, tolerance: 4.404e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.473e+00, tolerance: 4.639e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.131e+01, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.705e+00, tolerance: 3.831e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.381e+00, tolerance: 4.458e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.030e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.496e-02, tolerance: 4.816e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.915e-03, tolerance: 3.988e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.573e-02, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.478e+00, tolerance: 4.540e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.874e+00, tolerance: 4.816e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.082e+01, toleranc

...

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.314e+00, tolerance: 3.988e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.444e+00, tolerance: 4.631e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.279e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.009e+00, tolerance: 4.898e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.877e+00, tolerance: 3.831e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.783e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.069e+00, tolerance: 3.988e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e+01, tolerance: 4.631e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.755e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.639e-03, tolerance: 4.631e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.861e-02, tolerance: 4.540e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.812e-02, toleranc

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('lasso', Lasso())]),
             param_grid={'lasso__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'lasso__fit_intercept': [True, False],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler(), None]},
             scoring='r2')

In [62]:
best_model_lasso = grid_lasso.best_estimator_

In [63]:
y_pred_lasso = best_model_lasso.predict(X_test)

### Elastic Net Regression

In [66]:
from sklearn.linear_model import ElasticNet

In [67]:
elastic_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('elasticnet', ElasticNet())
])

In [68]:
elastic_param = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'poly__degree': [2,3,4],
    'poly__interaction_only': [False, True],
    'elasticnet__alpha': [0.001,0.01,0.1,1,10],
    'elasticnet__l1_ratio': [0.2,0.5,0.8],
    'elasticnet__fit_intercept': [True, False]
}

In [69]:
elastic_grid = GridSearchCV(elastic_pipe, elastic_param, cv=5, scoring='r2')

In [70]:
elastic_grid.fit(X_train, y_train)

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.471e-02, tolerance: 4.458e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.124e-02, tolerance: 4.404e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.540e-02, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.174e+00, tolerance: 4.458e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.874e+00, tolerance: 4.404e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.211e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.125e+00, tolerance: 4.458e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.242e+00, tolerance: 4.404e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.122e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.086e-01, tolerance: 4.639e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.203e+00, tolerance: 3.831e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.604e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.412e+00, tolerance: 4.639e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.078e+00, tolerance: 4.898e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.324e-02, toleranc

...

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.655e+00, tolerance: 5.056e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.275e-01, tolerance: 3.988e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.419e+00, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+01, tolerance: 4.816e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.981e+00, tolerance: 5.056e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.529e-03, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.286e-01, tolerance: 4.540e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.403e-03, tolerance: 4.816e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.242e-01, toleranc

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.679e-01, tolerance: 4.816e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.043e+00, tolerance: 5.056e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.739e-01, toleranc

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly', PolynomialFeatures()),
                                       ('elasticnet', ElasticNet())]),
             param_grid={'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'elasticnet__fit_intercept': [True, False],
                         'elasticnet__l1_ratio': [0.2, 0.5, 0.8],
                         'poly__degree': [2, 3, 4],
                         'poly__interaction_only': [False, True],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()]},
             scoring='r2')

In [71]:
best_model_elastic = elastic_grid.best_estimator_

In [72]:
y_pred_elastic = best_model_elastic.predict(X_test)

## Metrics

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, max_error
import math

In [23]:
def adjusted_r2_score(y_test, y_pred, X_test):
    r2 = r2_score(y_test, y_pred)  
    n = len(y_test)  
    p = X_test.shape[1]  
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)  
    return adjusted_r2

In [24]:
def mean_absolute_percentage_error(y_test, y_pred):
    return np.mean(np.abs((y_test-y_pred)/y_test))*100

### Linear Model

In [38]:
mae_lin = mean_absolute_error(y_test, y_lin_preds)
mse_lin = mean_squared_error(y_test, y_lin_preds)
rmse_lin = math.sqrt(mse_lin)
r2_lin = r2_score(y_test, y_lin_preds)
adjustr2_lin = adjusted_r2_score(y_test, y_lin_preds, X_test)
mape_lin = mean_absolute_percentage_error(y_test, y_lin_preds)
evs_lin = explained_variance_score(y_test, y_lin_preds)
maxe_lin = max_error(y_test, y_lin_preds)

In [39]:
adjustr2_lin

-0.36517410206337386

### Polynomial Regression

In [46]:
mae_pol = mean_absolute_error(y_test, y_pred_poly)
mse_pol = mean_squared_error(y_test, y_pred_poly)
rmse_pol = math.sqrt(mse_pol)
r2_pol = r2_score(y_test, y_pred_poly)
adjustr2_pol = adjusted_r2_score(y_test, y_pred_poly, X_test)
mape_pol = mean_absolute_percentage_error(y_test, y_pred_poly)
evs_pol = explained_variance_score(y_test, y_pred_poly)
maxe_pol = max_error(y_test, y_pred_poly)

In [47]:
adjustr2_pol

-0.3635162262094309

### Ridge Regression

In [54]:
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = math.sqrt(mse_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
adjustr2_ridge = adjusted_r2_score(y_test, y_pred_ridge, X_test)
mape_ridge = mean_absolute_percentage_error(y_test, y_pred_ridge)
evs_ridge = explained_variance_score(y_test, y_pred_ridge)
maxe_ridge = max_error(y_test, y_pred_ridge)

In [55]:
adjustr2_ridge

-0.3681077738310605

In [56]:
mape_ridge

67.71125875195064

### Lasso Regression

In [64]:
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = math.sqrt(mse_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
adjustedr2_lasso = adjusted_r2_score(y_test, y_pred_lasso, X_test)
mape_lasso = mean_absolute_percentage_error(y_test, y_pred_lasso)
evs_lasso = explained_variance_score(y_test, y_pred_lasso)
maxe_lasso = max_error(y_test, y_pred_ridge)

In [65]:
adjustedr2_lasso

-0.39709548347841905

### Elastic Net Regression 

In [73]:
mae_elastic = mean_absolute_error(y_test, y_pred_elastic)
mse_elastic = mean_squared_error(y_test, y_pred_elastic)
rmse_elastic = math.sqrt(mse_elastic)
r2_elastic = r2_score(y_test, y_pred_elastic)
adjustedr2_elastic = adjusted_r2_score(y_test, y_pred_elastic, X_test)
mape_elastic = mean_absolute_percentage_error(y_test, y_pred_elastic)
evs_elastic = explained_variance_score(y_test, y_pred_elastic)
maxe_elastic = max_error(y_test, y_pred_elastic)

In [74]:
adjustedr2_elastic

-0.4036457074736839

## Alignment of All Metrics

In [75]:
metrics_dict = {
    'Linear': [
        mae_lin, mse_lin, rmse_lin, r2_lin,
        adjustr2_lin, mape_lin, evs_lin, maxe_lin],
    'Polynomial': [
        mae_pol, mse_lin, rmse_pol, r2_pol,
        adjustr2_pol, mape_pol, evs_pol, maxe_pol
    ],
    'Ridge': [
        mae_ridge, mse_ridge, rmse_ridge, r2_ridge, 
        adjustr2_ridge, mape_ridge, evs_ridge, maxe_ridge
    ],
    'Lasso': [
        mae_lasso, mse_lasso, rmse_lasso, r2_lasso, 
        adjustedr2_lasso, mape_lasso, evs_lasso, maxe_lasso
    ],
    'ElasticNet': [
        mae_elastic, mse_elastic, rmse_elastic, r2_elastic, 
        adjustedr2_elastic, mape_elastic, evs_elastic, maxe_elastic
    ]
}

In [76]:
metrics_df = pd.DataFrame(metrics_dict, 
                         index=['MAE', 'MSE', 'RMSE',
                               'R2', 'Adjusted R2',
                               'MAPE', 'EVS', 'MAXError'])

In [77]:
metrics_df

,Linear,Polynomial,Ridge,Lasso,ElasticNet
MAE,0.135396,0.135734,0.138101,0.140479,0.140715
MSE,0.034827,0.034827,0.034902,0.035641,0.035808
RMSE,0.186619,0.186506,0.186820,0.188789,0.189231
R2,-0.023881,-0.022637,-0.026081,-0.047822,-0.052734
Adjusted R2,-0.365174,-0.363516,-0.368108,-0.397095,-0.403646
MAPE,67.872019,76.711539,67.711259,74.309405,74.340047
EVS,0.027045,0.025087,0.038345,0.020140,0.017657
MAXError,0.741789,0.732250,0.758230,0.758230,0.774848
